In [ ]:
import os
import sys
from pathlib import Path

# Navigate to project root (equivalent to cd ..)
project_dir = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_dir)

# Add src directory to Python path for imports
src_dir = project_dir / "src"
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Set environment for dev testing
os.environ['REPORT_ENV'] = 'prod'

In [ ]:
import src.config
from deltalake import DeltaTable
from pathlib import Path
import pandas as pd
from datetime import datetime, timezone


In [ ]:
TABLE_PATH = src.config.SILVER / "account"

In [ ]:
history = DeltaTable(TABLE_PATH).history()
history_df = pd.DataFrame(history)


In [ ]:
def convert_ts(ts):
    if len(str(ts)) == 13:
        return pd.to_datetime(ts, unit='ms', utc=True)
    else:
        return pd.to_datetime(ts, unit='s', utc=True)

history_df['readable_time'] = history_df['timestamp'].apply(convert_ts)

In [ ]:
history_df['eastern_time'] = history_df['readable_time'].dt.tz_convert('America/New_York')

In [ ]:
history_df

In [ ]:
df = DeltaTable(TABLE_PATH, version=8).to_pandas()

In [ ]:
df['eastern_time'] = df['load_timestamp_utc'].dt.tz_convert('America/New_York')

In [ ]:
df